In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_qa_master_tr.csv")
print("Toplam satır:", len(df))
print(df.columns)


Toplam satır: 614
Index(['question_id', 'question', 'answer', 'question_type',
       'source_filenames', 'source_sections', 'source_chunk_ids',
       'is_impossible', 'dataset_split', 'question_tr', 'answer_tr'],
      dtype='object')


In [ ]:
train_df = df[(df["dataset_split"] == "train") & (df["is_impossible"] == False)].copy()
print("Eğitime uygun satır:", len(train_df))


Eğitime uygun satır: 470


In [ ]:
def build_input(q: str) -> str:
    q = str(q).strip()
    return f"Soru: {q}\nCevap:"

def clean_answer(a: str) -> str:
    a = str(a).strip()
    # Çok satırlıysa tek satıra indir
    return " ".join(a.split())

train_df["input_text"]  = train_df["question_tr"].apply(build_input)
train_df["target_text"] = train_df["answer_tr"].apply(clean_answer)

train_df[["input_text", "target_text"]].head()


,input_text,target_text
0,Soru: Politikaya göre kesinlikle yasak olan ba...,"Politika, telif hakkıyla korunan materyalin iz..."
1,Soru: Bu politikayı ihlal eden kişiler açısınd...,"Bu politikaya uyulmaması, zorunlu tazeleme eği..."
2,"Soru: Politikaya göre, tüm kablosuz ağlar için...",Kablosuz ağların güvenliğini sağlamak için tüm...
4,Soru: Kuruluşun Bulut Hizmeti Sağlayıcılarının...,"Politika, yetkili CSP'lerin veya SaaS sağlayıc..."
6,Soru: SAF-13'ün amacı nedir?\nCevap:,"SAF-13, kuruluşun siber güvenlik önlemi belgel..."


In [ ]:
for i in range(3):
    print("---- ÖRNEK", i, "----")
    print(train_df["input_text"].iloc[i])
    print("→", train_df["target_text"].iloc[i])
    print()


---- ÖRNEK 0 ----
Soru: Politikaya göre kesinlikle yasak olan bazı faaliyet örnekleri nelerdir?
Cevap:
→ Politika, telif hakkıyla korunan materyalin izinsiz kopyalanması, lisanssız yazılım kurulumu ve verilere veya hesaplara yetkisiz erişim de dahil olmak üzere fikri mülkiyet haklarının ihlal edilmesini yasaklar.Ayrıca ihracat kontrol yasalarını ihlal edecek şekilde yazılım veya teknolojinin ihraç edilmesini de yasaklar.

---- ÖRNEK 1 ----
Soru: Bu politikayı ihlal eden kişiler açısından potansiyel sonuçlar nelerdir?
Cevap:
→ Bu politikaya uyulmaması, zorunlu tazeleme eğitimi ve yazılı uyarılardan, uzaktan erişim ayrıcalıklarının geçici olarak askıya alınmasına ve ciddi durumlarda iş akdinin veya sözleşmeden doğan yükümlülüklerin feshine kadar değişen disiplin cezalarıyla sonuçlanabilir.

---- ÖRNEK 2 ----
Soru: Politikaya göre, tüm kablosuz ağlar için hangi özel güvenlik önleminin uygulanması gerekiyor ve bunun amacı nedir?
Cevap:
→ Kablosuz ağların güvenliğini sağlamak için tüm kablo

In [ ]:
out_path = "bilgi_guvenligi_qa_for_finetune.csv"
train_df[["input_text", "target_text"]].to_csv(out_path, index=False)
print("Kaydedildi:", out_path)


Kaydedildi: bilgi_guvenligi_qa_for_finetune.csv


In [ ]:
import pandas as pd
import json

# CSV dosyasını oku
df = pd.read_csv("https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_qa_master_tr.csv")

# Sadece train + cevaplanabilir satırlar
train_df = df[(df["dataset_split"] == "train") & (df["is_impossible"] == False)].copy()

# Soru–cevap formatı
def build_input(q: str) -> str:
    q = str(q).strip()
    return f"Soru: {q}\nCevap:"

def clean_answer(a: str) -> str:
    a = str(a).strip()
    return " ".join(a.split())

train_df["input"]  = train_df["question_tr"].apply(build_input)
train_df["output"] = train_df["answer_tr"].apply(clean_answer)

# JSONL dosyasını oluştur
out_path = "bilgi_guvenligi_qa_for_finetune.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for _, row in train_df.iterrows():
        json_obj = {
            "input": row["input"],
            "output": row["output"]
        }
        f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")

print("JSONL dosyası oluşturuldu:", out_path)
print("Toplam satır:", len(train_df))


JSONL dosyası oluşturuldu: bilgi_guvenligi_qa_for_finetune.jsonl
Toplam satır: 470


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/FINE-TUNE/QA_dataset.csv"

df = pd.read_csv(url)

print("Toplam satır sayısı:", len(df))
print("Tam satır duplicate sayısı:", df.duplicated().sum())
print("qa_id duplicate sayısı:", df["qa_id"].duplicated().sum())


Toplam satır sayısı: 1042
Tam satır duplicate sayısı: 0
qa_id duplicate sayısı: 0


In [ ]:
import pandas as pd
import json

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/FINE-TUNE/QA_dataset.csv"
df = pd.read_csv(url)

output_path = "QA_dataset_full.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        record = {
            "qa_id": row["qa_id"],
            "input_text": row["input_text"],
            "target_text": row["target_text"],
            "source_standard": row["source_standard"],
            "source_doc": row["source_doc"],
            "control_id": row["control_id"],
            "question_type": row["question_type"],
            "difficulty": row["difficulty"],
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("Kaydedildi:", output_path)


Kaydedildi: QA_dataset_full.jsonl


In [ ]:
import json
import pandas as pd

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/FINE-TUNE/QA_dataset.csv"
df = pd.read_csv(url)

output_path = "gemma_qa_messages.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        record = {
            "messages": [
                {
                    "role": "user",
                    "content": row["input_text"]
                },
                {
                    "role": "model",
                    "content": row["target_text"]
                }
            ]
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("JSONL kaydedildi:", output_path)


JSONL kaydedildi: gemma_qa_messages.jsonl


In [ ]:
import json
import requests

# URL'den veri al
url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/FINE-TUNE/QA_dataset_gemma.jsonl"
response = requests.get(url)
response_text = response.text.splitlines()

# Çıkış dosyası
output_path = "/content/QA_dataset_Vson.jsonl"

seen_pairs = set()

with open(output_path, "w", encoding="utf-8") as fout:

    for line in response_text:
        if not line.strip():
            continue

        ex = json.loads(line)
        msgs = ex["messages"]

        user_msg  = next(m["content"] for m in msgs if m["role"] == "user")
        model_msg = next(
            m["content"] for m in msgs
            if m["role"] in ("model", "assistant")
        )

        user_msg_clean  = user_msg.strip()
        model_msg_clean = model_msg.strip()

        pair = (user_msg_clean, model_msg_clean)
        if pair in seen_pairs:
            continue
        seen_pairs.add(pair)

        text = (
            "<start_of_turn>user\n"
            + user_msg_clean
            + "\n<end_of_turn>\n"
            "<start_of_turn>model\n"
            + model_msg_clean
            + "\n<end_of_turn>"
        )

        json.dump({"text": text}, fout, ensure_ascii=False)
        fout.write("\n")

print("✅ Dönüşüm tamamlandı!")
print("📄 Yeni dosya:", output_path)
print("🧮 Toplam örnek:", len(seen_pairs))


✅ Dönüşüm tamamlandı!
📄 Yeni dosya: /content/QA_dataset_Vson.jsonl
🧮 Toplam örnek: 1042
